In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
%matplotlib inline
plt.rcParams['figure.figsize']=10,5

IMPACT OF COVID-19 ON TEAM PERFORMANCE (BY CALCULATING THE WIN RATE)

In [ ]:
match_df= pd.read_csv('matches.csv')

In [ ]:
match_data = match_df[['date','team','opponent', 'gf', 'ga','result']]
match_data

,date,team,opponent,gf,ga,result
0,2022-08-05,Arsenal,Crystal Palace,2.0,0.0,W
1,2022-08-13,Arsenal,Leicester City,4.0,2.0,W
2,2022-08-20,Arsenal,Bournemouth,3.0,0.0,W
3,2022-08-27,Arsenal,Fulham,2.0,1.0,W
4,2022-08-31,Arsenal,Aston Villa,2.0,1.0,W
...,...,...,...,...,...,...
3267,2019-04-13,Huddersfield Town,Tottenham,0.0,4.0,L
3268,2019-04-20,Huddersfield Town,Watford,1.0,2.0,L
3269,2019-04-26,Huddersfield Town,Liverpool,0.0,5.0,L
3270,2019-05-05,Huddersfield Town,Manchester Utd,1.0,1.0,D


In [ ]:
pd.set_option('mode.chained_assignment', None)
match_data['TeamWin'] = match_data['result'].apply(lambda x: 1 if x is 'W' else 0)
match_data['TeamLoss'] = match_data['result'].apply(lambda x: 1 if x is 'L' else 0)
match_data['TeamDraw'] = match_data['result'].apply(lambda x: 1 if x is 'D' else 0)

match_data['OppWin'] = match_data['result'].apply(lambda x: 1 if x is 'L' else 0)
match_data['OppLoss'] = match_data['result'].apply(lambda x: 1 if x is 'W' else 0)
match_data['OppDraw'] = match_data['result'].apply(lambda x: 1 if x is 'D' else 0)
match_data = match_data.drop('result', axis=1)
match_data

,date,team,opponent,gf,ga,TeamWin,TeamLoss,TeamDraw,OppWin,OppLoss,OppDraw
0,2022-08-05,Arsenal,Crystal Palace,2.0,0.0,1,0,0,0,1,0
1,2022-08-13,Arsenal,Leicester City,4.0,2.0,1,0,0,0,1,0
2,2022-08-20,Arsenal,Bournemouth,3.0,0.0,1,0,0,0,1,0
3,2022-08-27,Arsenal,Fulham,2.0,1.0,1,0,0,0,1,0
4,2022-08-31,Arsenal,Aston Villa,2.0,1.0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
3267,2019-04-13,Huddersfield Town,Tottenham,0.0,4.0,0,1,0,1,0,0
3268,2019-04-20,Huddersfield Town,Watford,1.0,2.0,0,1,0,1,0,0
3269,2019-04-26,Huddersfield Town,Liverpool,0.0,5.0,0,1,0,1,0,0
3270,2019-05-05,Huddersfield Town,Manchester Utd,1.0,1.0,0,0,1,0,0,1


In [ ]:
# splitting data into preCOVID and postCOVID
preCOVID = match_data.loc[match_data['date']<'2021-03-01']
postCOVID = match_data.loc[match_data['date']>'2022-04-01']

In [ ]:
preCOVID

,date,team,opponent,gf,ga,TeamWin,TeamLoss,TeamDraw,OppWin,OppLoss,OppDraw
992,2020-09-21,Manchester City,Wolves,3.0,1.0,1,0,0,0,1,0
993,2020-09-27,Manchester City,Leicester City,2.0,5.0,0,1,0,1,0,0
994,2020-10-03,Manchester City,Leeds United,1.0,1.0,0,0,1,0,0,1
995,2020-10-17,Manchester City,Arsenal,1.0,0.0,1,0,0,0,1,0
996,2020-10-24,Manchester City,West Ham,1.0,1.0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
3267,2019-04-13,Huddersfield Town,Tottenham,0.0,4.0,0,1,0,1,0,0
3268,2019-04-20,Huddersfield Town,Watford,1.0,2.0,0,1,0,1,0,0
3269,2019-04-26,Huddersfield Town,Liverpool,0.0,5.0,0,1,0,1,0,0
3270,2019-05-05,Huddersfield Town,Manchester Utd,1.0,1.0,0,0,1,0,0,1


In [ ]:
postCOVID

,date,team,opponent,gf,ga,TeamWin,TeamLoss,TeamDraw,OppWin,OppLoss,OppDraw
0,2022-08-05,Arsenal,Crystal Palace,2.0,0.0,1,0,0,0,1,0
1,2022-08-13,Arsenal,Leicester City,4.0,2.0,1,0,0,0,1,0
2,2022-08-20,Arsenal,Bournemouth,3.0,0.0,1,0,0,0,1,0
3,2022-08-27,Arsenal,Fulham,2.0,1.0,1,0,0,0,1,0
4,2022-08-31,Arsenal,Aston Villa,2.0,1.0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
987,2022-04-30,Norwich City,Aston Villa,0.0,2.0,0,1,0,1,0,0
988,2022-05-08,Norwich City,West Ham,0.0,4.0,0,1,0,1,0,0
989,2022-05-11,Norwich City,Leicester City,0.0,3.0,0,1,0,1,0,0
990,2022-05-15,Norwich City,Wolves,1.0,1.0,0,0,1,0,0,1


In [ ]:
#CALCULATING WIN RATE FOR TEAMS PRE AND POST COVID-19

# preCOVID 

team_data = preCOVID.groupby('team').agg({'team': 'count', 'gf': 'mean', 'ga' : 'mean', 'TeamWin' : 'sum', 'TeamDraw' : 'sum', 'TeamLoss' : 'sum'})
team_data.columns = ['# pre-COVID matches', 'Ave. Goals For', 'Ave. Goals Against', 'preCOVID Won', 'preCOVID Drawn', 'preCOVID Lost']

team_data['preCOVID WinRate'] = team_data['preCOVID Won']/team_data['# pre-COVID matches'] * 100
team_data = team_data.sort_values('preCOVID WinRate', ascending = False)

# postCOVID 

team_data2 = postCOVID.groupby('team').agg({'team': 'count', 'gf': 'mean', 'ga' : 'mean', 'TeamWin' : 'sum', 'TeamDraw' : 'sum', 'TeamLoss' : 'sum'})
team_data2.columns = ['# post-COVID matches', 'Ave. Goals For', 'Ave. Goals Against', 'postCOVID Won', 'postCOVID Drawn', 'postCOVID Lost']

team_data2['postCOVID WinRate'] = team_data2['postCOVID Won']/team_data2['# post-COVID matches'] * 100
data2 = team_data2.sort_values('postCOVID WinRate', ascending = False)

finalData = pd.concat([team_data,team_data2], axis=1)
finalData

,# pre-COVID matches,Ave. Goals For,Ave. Goals Against,preCOVID Won,preCOVID Drawn,preCOVID Lost,preCOVID WinRate,# post-COVID matches,Ave. Goals For,Ave. Goals Against,postCOVID Won,postCOVID Drawn,postCOVID Lost,postCOVID WinRate
team,,,,,,,,,,,,,,
Manchester City,102.0,2.441176,0.725490,77.0,10.0,15.0,75.490196,20.0,3.350000,0.950000,15.0,4.0,1.0,75.000000
Liverpool,102.0,2.166667,0.872549,74.0,17.0,11.0,72.549020,20.0,2.050000,0.950000,11.0,6.0,3.0,55.000000
Chelsea,102.0,1.696078,1.156863,53.0,23.0,26.0,51.960784,21.0,1.666667,1.190476,10.0,6.0,5.0,47.619048
Manchester United,102.0,1.803922,1.196078,51.0,29.0,22.0,50.000000,20.0,1.250000,1.650000,8.0,4.0,8.0,40.000000
Tottenham Hotspur,101.0,1.673267,1.118812,50.0,19.0,32.0,49.504950,21.0,2.142857,0.857143,13.0,4.0,4.0,61.904762
Leicester City,102.0,1.598039,1.166667,48.0,19.0,35.0,47.058824,23.0,1.782609,1.608696,7.0,6.0,10.0,30.434783
Arsenal,102.0,1.598039,1.235294,46.0,25.0,31.0,45.098039,21.0,2.000000,1.333333,14.0,1.0,6.0,66.666667
Leeds United,26.0,1.653846,1.692308,11.0,2.0,13.0,42.307692,19.0,1.105263,1.578947,4.0,6.0,9.0,21.052632
Everton,100.0,1.350000,1.350000,40.0,23.0,37.0,40.000000,23.0,1.086957,1.347826,7.0,6.0,10.0,30.434783
